<a href="https://colab.research.google.com/github/GoKarolis/ML_training_project/blob/main/kg_red_wine_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.preprocessing import LabelEncoder #, StandardScaler

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# wine_data == data
data = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/winequality-red.csv')

# 1. Exploratory data analysis

# **1.1. Shape**

In [ ]:
data.shape

(1599, 12)

# **1.2. Descriptive statistics**

In [ ]:
data.head(20)

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
def display_all(data):
    with pd.option_context("display.max_rows", 10000, "display.max_columns", 10000): 
        display(data)

display_all(data.describe(include='all').T)

# **1.3. Data types and more information about the data**

In [ ]:
data.dtypes

In [ ]:
data.isna().sum()

In [ ]:
data['quality'].value_counts().sort_index(ascending=False)

# 3. Pre-processing

In [ ]:
bins = (2, 6.5, 8)
group_names = ['bad', 'good']
data['quality'] = pd.cut(data['quality'], bins = bins, labels = group_names)

In [ ]:
label_quality = LabelEncoder()

data['quality'] = label_quality.fit_transform(data['quality'])

X = data.drop('quality', axis = 1)
y = data['quality']


# Train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
def split_vals(a,n): return a[:n], a[n:]
n_valid = 200
n_trn = len(data)-n_valid
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)
raw_train, raw_valid = split_vals(df_raw, n_trn)

**Out-of-bag score**

In [ ]:
"""y = data.quality
X = data
X_train,X_test,y_train,y_test = train_test_split(data.drop('quality',axis=1),data['quality'],test_size=0.25,random_state=42)


m = RandomForestRegressor(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)"""

In [ ]:
"""valid_fraction = 0.1
valid_size = int(len(data) * valid_fraction)

x_train = data[:-2 * valid_size]
x_valid = data[-2 * valid_size:-valid_size]
x_test = data[-valid_size:]
"""

In [ ]:
X = data.drop('quality',axis=1)
y = data['quality']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# k-Nearest Neighbour

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn import metrics
from sklearn.metrics import classification_report

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print(classification_report(y_test,y_pred))



"""neighbors = np.arange(1,9)
train_accuracy =np.empty(len(neighbors))
valid_accuracy = np.empty(len(neighbors))

for i,k in enumerate(neighbors):
    #Setup a knn classifier with k neighbors
    knn = KNeighborsClassifier(n_neighbors=k)
    
    #Fit the model
    knn.fit(X_train, y_test)
    
    #Compute accuracy on the training set
    train_accuracy[i] = knn.score(X_train, y_train)
    
    #Compute accuracy on the test set
    test_accuracy[i] = knn.score(X_test, y_test)   """

              precision    recall  f1-score   support

           0       0.88      0.97      0.92       273
           1       0.53      0.21      0.30        47

    accuracy                           0.86       320
   macro avg       0.70      0.59      0.61       320
weighted avg       0.83      0.86      0.83       320



'neighbors = np.arange(1,9)\ntrain_accuracy =np.empty(len(neighbors))\nvalid_accuracy = np.empty(len(neighbors))\n\nfor i,k in enumerate(neighbors):\n    #Setup a knn classifier with k neighbors\n    knn = KNeighborsClassifier(n_neighbors=k)\n    \n    #Fit the model\n    knn.fit(X_train, y_test)\n    \n    #Compute accuracy on the training set\n    train_accuracy[i] = knn.score(X_train, y_train)\n    \n    #Compute accuracy on the test set\n    test_accuracy[i] = knn.score(X_test, y_test)   '

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

X = data.drop('quality',axis=1).values
y = data['quality'].values

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

"""neighbors = np.arange(1,9)
train_accuracy =np.empty(len(neighbors))
valid_accuracy = np.empty(len(neighbors))

for i,k in enumerate(neighbors):
    #Setup a knn classifier with k neighbors
    knn = KNeighborsClassifier(n_neighbors=k)
    
    #Fit the model
    knn.fit(x_train, y_test)
    
    #Compute accuracy on the training set
    train_accuracy[i] = knn.score(x_train, y_train)
    
    #Compute accuracy on the test set
    test_accuracy[i] = knn.score(x_test, y_test) 
    


#X = data.data
#y = data.target

#y = data.quality
#data_features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'total sulfur dioxide']
#X = data[data_features]

#x_train,x_test,y_train,y_test=train_test_split(data.drop('quality',axis=1),data['quality'],test_size=0.25,random_state=42)


k = 1
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(x_train,y)

yhat = neigh.predict(x_valid)
yhat[0:5]

# Jaccard index

print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(x_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_valid, yhat))
"""

# Decision Tree

In [ ]:
#from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error


y = data.quality
wine_features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'total sulfur dioxide']
X = data[wine_features]

#train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]

scores = {leaf_size: get_mae(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}
best_tree_size = min(scores, key=scores.get)

final_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state=1)

final_model.fit(X, y)




val_predictions = final_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE: {:,.0f}".format(val_mae))

# Random forest

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred=rf.predict(X_test)

print(classification_report(y_test,y_pred))

# Logistic regression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_predict = lr.predict(X_test)

print(classification_report(y_test,lr_predict))

Requirements
    Do exploratory data analysis.
    Do feature engineering.
    Do feature preprocessing.
    Build a machine learning model.
    Explore your model results.
Bonus Points
    Move your final model to a Python module.
    Compare multiple machine learning models.
    Do hyperparameter tuning.
    Deploy the model on your own machine.
    Deploy the model on external service.


 

7. If missing values are present then how to impute them?
 

For various scenarios, while dealing with data you will come across real-world data which will have missing values like nan values, -, blanks. The basic approach to deal with such a situation is to drop/ remove the entire row or column which contains missing values. But dropping is not advisable because there will be a loss of data as well which can result in important parts of the data being removed. So, to deal with such things there are different methods used to impute the missing values. 

 

There are two ways by which missing values can be imputed: first is called univariate imputation and the other one is multivariate imputation. 

 

Univariate imputation is a type of imputation which imputes missing values considering only the non-missing values in that feature dimension. (e.g. impute.SimpleImputer). 

 

On the other hand, a multivariate imputer imputes the missing values considering all available features dimension.(e.g. impute.IterativeImputer). You can read more about imputing missing values here.

 

 

8. Encoding categorical features 
 

Often it is seen that we do not have continuous values in our features. There are sometimes categorical values. And the system cannot understand such values so there is a need to convert them to continuous numerical values. 

 

As seen in the below iris data frame we have classes as categorical features which are - ‘Iris-setosa’, ‘Iris-versicolor’, ‘Iris-virginica’.

 

How to encode them?
 
 
Unique values in class column of the iris data-set which contains categorical values.

 

There are several different techniques which are used to encode categorical values which are stated below:

 

a) LabelEncoder() - It is a function present in the scikit- learn library of python which is used to convert categorical values in numerical values.

 

Hands-on implementation of encoding using LabelEncoder.

 

Here we have imported LabelEncoder from sklearn.preprocessing followed by initialising of the object through which we will use the label encoder. We have made an object called “LE”. Then we have transformed our class column by using the LE.fit_transform function & printed the transformed class which is now [0,1,2]. It has given the values to Iris-setosa - 0, Iris-versicolor  - 1 , Iris-virginica - 2.

 

b) get_dummies() - Converts categorical features into dummy variables. 

 

c) OneHotEncoder() - Array-like of integers or strings is the required input for this encoder. The features are encoded using a one-hot encoding scheme. The result is a binary column for each category and reverts a sparse matrix. 

 

 

9. Standardization of data 
 

Standardization of data is a major important step that is required for machine learning algorithms to give good results. There are different scaling functions present in the preprocessing module of sci-kit learn. If data is not scaled and is passed to the algorithm the result might be wrong due to wrongly distributed data.

 

Why is it important to scale the data?
 

It is usually seen that we ignore checking the shape of the distribution and change the data to be centered. That is done by removing the mean values of each column and then scaling it by dividing non-constant columns by their standard deviation.

 

Different functions that are used by algorithms to learn to assume that all the desired features are centered as zero and also their variance is in the same structure. If any of the features have a higher proportion than all other features it may dominate the function for learning algorithm and does not allow learning from other features as required. 

 

a) Scale :  present in the pre-processing module gives a fast and effective way to do this operation on a single array-like data:

 

Hand-on implementation of scaling using scale.

 

X_scaled has now unit variance and zero mean as you can see in the below image.

 

Scaled data has zero mean and unit variance.

 

b) The pre-processing module also has different other classes like StandardScaler that are used in scaling the data that is converting the mean to be zero and standard deviation to be unit on training data which can be further used in test data as well. Such class can also be used in building pipelines also.

 

The code implementation of standard scaler is shown below.

 

Code implementation of Standard scaler.

 

c) Scaling features to a range:  There are other methods also to scale data within a respective range that is a min values and max value. It mainly ranges between 0 and 1. You can use MinMaxScaler or MaxAbsScaler for scaling the data respectively.

 

d) Scaling sparse data: Centering the scatter data would result in knock-down of sparsity structure of data thus it is not advisable to do. MinMaxScaler and MaxAbs scaler were introduced to scale the sparse data. Scalar often accepts both CSR (Compressed Sparse Rows) & also CSC (Compressed Sparse Columns). If there is any other different sparse input then it is converted to Compressed Sparse Rows. To take care of the memory it is advisable to convert it in CSR and CSC representation.

 

e) Scaling data with presence of outliers : If the data has outliers in it then scaling that sort of data using mean and variance is not a good approach. You can use robust_scale & Robust_Scaler as drop in substitution.

 

10. Normalization of data 
 

It is the process of scaling each sample to have a unit standard. These types of techniques are much more effective if you are computing the similarity between different pairs of samples or to use a quadratic form like a dot product.This is the base of models used in text classifications. 

 

There is a function in the pre-processing module that is normalized which provides a good way to execute such operations on single array-like data by using l1 or l2 standards. Implementation of normalizing data using normalize is shown in the below image.

 

Code to normalize the data using Normalize function.

 

Pre-processing module also has another class that is called a normalizer that executes the similar operations using the transformer API. This class can also be used in the initial stage of pipeline. Implementation of normalizer is shown below in the image.

 

Code to normalize the data using normalizer function.

 

If you want to look for code implementation of EDA discussed above you can refer to the GitHub link here. It contains a jupyter file and both the datasets which are used. 


# XGBOOST

In [ ]:
xgb = XGBClassifier(max_depth=3,n_estimators=200,learning_rate=0.5)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
# print(accuracy_score(y_test,y_pred))



rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred=rf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94       273
           1       0.74      0.53      0.62        47

    accuracy                           0.90       320
   macro avg       0.83      0.75      0.78       320
weighted avg       0.90      0.90      0.90       320



In [ ]:
xgb.imp <- xgb.importance(colnames(train[, !colnames(valid) %in% c("quality")]), model = xgb_model)

xgb.ggplot.importance(importance_matrix = xgb.imp)